In [2]:
import openai
import pinecone 
import tqdm
import requests
import json
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity


In [3]:
API_KEY = "XXX"
RESOURCE_ENDPOINT = 'XXX'

pinecone.init(api_key="XXX")
index_name = 'item-embeddings'

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"
r = requests.get(url, headers={"api-key": API_KEY})
print(r.text)  # see the models deployed in your account 



{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt-35-turbo-ozguler",
      "status": "succeeded",
      "created_at": 1679424805,
      "updated_at": 1679424805,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-003",
      "owner": "organization-owner",
      "id": "text-davinci-003-ozguler",
      "status": "succeeded",
      "created_at": 1679425352,
      "updated_at": 1679425352,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "text-embedding-ada-002",
      "status": "succeeded",
      "created_at": 1680255606,
      "updated_at": 1680255606,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scal

In [17]:
### text_columns = ['About Product', 'Product Specification', 'Technical Details', 'Ingredients', 'Direction To Use', 'Product Description']

In [6]:
import pandas as pd
df = pd.read_csv("./netflix_titles.csv")
df = df.loc[:, ["title", "description"]] 
df.shape
df.head()


,title,description
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm..."
1,Blood & Water,"After crossing paths at a party, a Cape Town t..."
2,Ganglands,To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,In a city of coaching centers known to train I...


In [7]:
import re
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    return s

df["description"] = list(map(lambda x: normalize_text(x), df["description"].values))
df.head()
#print(df["description"].values[1])

,title,description
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm..."
1,Blood & Water,"After crossing paths at a party, a Cape Town t..."
2,Ganglands,To protect his family from a powerful drug lor...
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo..."
4,Kota Factory,In a city of coaching centers known to train I...


In [8]:
#df["completions"][1:5] = df["description"].apply(lambda x : openai.ChatCompletion.create(messages = x, engine = 'gpt-35-turbo'))
df["embeddings"] = df["description"].apply(lambda x : get_embedding(x, engine = 'text-search-curie-doc-001'))

In [13]:
df["embeddings"]
df["embeddings"].shape
df["embeddings"][0]


[0.0043516214936971664,
 0.0022802685853093863,
 -0.002079661935567856,
 0.0006783828721381724,
 0.0040999725461006165,
 -0.004427590873092413,
 -0.012326032854616642,
 -0.01153784990310669,
 -0.007554205600172281,
 0.0165898147970438,
 0.0052893683314323425,
 -0.009068845771253109,
 -0.02506989613175392,
 0.0007970850565470755,
 0.019562115892767906,
 -0.031755201518535614,
 -0.00652387086302042,
 -0.012952780351042747,
 0.0022399097215384245,
 0.010217882692813873,
 -0.013987863436341286,
 -0.004565285053104162,
 -0.017425477504730225,
 0.009182799607515335,
 -0.008916907012462616,
 0.013275650329887867,
 0.008199945092201233,
 -0.026304399594664574,
 -0.01684621162712574,
 -0.010151408612728119,
 -0.009197043254971504,
 -0.010645209811627865,
 0.010084935463964939,
 0.06483986973762512,
 0.02991294488310814,
 0.0272540170699358,
 0.011376415379345417,
 0.012648902833461761,
 -0.007986281998455524,
 -0.08918805420398712,
 -0.02142336592078209,
 -0.0008089552284218371,
 -0.00983803533

In [9]:
# search through the descriptions for a specific movie title / descripton
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text-search-curie-query-001"
    )

    df["similarities"] = df.embeddings.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


#res = search_docs(df, "A young Han Solo tries to settle an old score with the help of his new buddy Chewbacca, a crew of space smugglers and a cunning old friend.", top_n=10)

In [ ]:
title = "Star Trek"
description = df.loc[df["title"] == title, "description"].iloc[0]
res = search_docs(df, description, top_n=15)

In [64]:
df

,title,description,embeddings,similarities
0,Dick Johnson Is Dead,"As her father nears the end of his life, filmm...","[0.0043516214936971664, 0.0022802685853093863,...",0.290345
1,Blood & Water,"After crossing paths at a party, a Cape Town t...","[-0.02388826385140419, 0.0029224504251033068, ...",0.266189
2,Ganglands,To protect his family from a powerful drug lor...,"[-0.034246984869241714, 0.010515750385820866, ...",0.326060
3,Jailbirds New Orleans,"Feuds, flirtations and toilet talk go down amo...","[-0.02306009829044342, -0.008633919060230255, ...",0.248097
4,Kota Factory,In a city of coaching centers known to train I...,"[-0.030220653861761093, 0.009630915708839893, ...",0.261389
...,...,...,...,...
8802,Zodiac,"A political cartoonist, a crime reporter and a...","[-0.022079553455114365, 0.00714001152664423, -...",0.274146
8803,Zombie Dumb,"While living alone in a spooky town, a young g...","[-0.025093426927924156, -0.007184464950114489,...",0.294525
8804,Zombieland,Looking to survive in a world taken over by zo...,"[-0.02202506549656391, -0.007789113558828831, ...",0.304413
8805,Zoom,"Dragged from civilian life, a former superhero...","[-0.01231981161981821, -0.01197921670973301, 0...",0.301935
